# Heart Disease Classification
Phase 3 Project by Bella Scribner
- Flex
- Morgan Jones
- Date of Project Review:
- Blog: 

# Intro + Data Exploration
- The Business + Buisness Problem
- Overview of project goals/what want to accomplish in order to 'solve' or address the business problem

- Data Exploration
    - What data set working with
    - Explination of the columns (identify target)
    - .describe, .info, .value counts of target (introduce idea of maybe needing SMOTE)
    - Missing Data? --> how might handle based on what kind of data is missing (and how much)

# Data Pre-processing

- Train Test Split
- If missing values --> handle + explian why chosen method(s)
- Categorical columns -- > encode (don't forget handel unknown = ignore)
- SMOTE --> see lecture! imblear's pipelines!!

### NOTE ---> wondering if I should do this in a pipeline at first? Or for simple, initial model, do 'by hand' preprocessing and then convert into pipeline???

## Important to choose what types of models going to try!!
- some models require scaling of numberical values --> need to get into data and decide which models seem most appropriate and if some of those models require scaling --> add to preprocessing/pipeline

THis is where it will get a little tricky to stay organized. Different models need different preprocessing --> should just create pipeline for each model I suppose to keep it as clean as possible.  Plus can use pipeline + grid search (+cross validation) for later on...

# Model Iteration (3 minimum)
- Go through the different models
    - why chose each model?
    - quick evaluation of each model
    - iterate through (hopefully improving) each model
- Evaluation:
    - appropriate metric for business problem!! indicate reasoning for which metric(s) using for each model DO NOT try to use all at once

# Chose Final Model
- choose final model
- explain why
- findings + recommendations
    - model features - which ones important to model?
    - how well can model predict target?
    - contexts/situations where the predictions made by model would and would not be useful for your stakeholder and business problem
    - what should stakeholders DO with this information??
    - suggestions for how the buisness might modify certain input variables to achieve certain target results
    - ETC...
    - Limitations/Uncertainty in project?